<a href="https://colab.research.google.com/github/Suryansh1089/Sarcasm-Detection/blob/main/Sarcasm%20Detection%20using%20Sequential%20and%20Convolutional%20Neural%20network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Run this to ensure TensorFlow 2.x is used
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

In [2]:
!pip install jsonlines
import jsonlines
import tensorflow as tf

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [3]:
vocab_size = 1000000
embedding_dim = 16
max_length = 100
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"
training_size =2500


In [5]:
import pandas as pd


In [6]:
type('twitter_train.jsonl')

str

In [9]:
twitter_train=pd.read_json('twitter_train.jsonl',lines=True)

In [10]:
twitter_train.head()

,label,response,context
0,SARCASM,@USER @USER @USER I don't get this .. obviousl...,[A minor child deserves privacy and should be ...
1,SARCASM,@USER @USER trying to protest about . Talking ...,[@USER @USER Why is he a loser ? He's just a P...
2,SARCASM,@USER @USER @USER He makes an insane about of ...,[Donald J . Trump is guilty as charged . The e...
3,SARCASM,@USER @USER Meanwhile Trump won't even release...,[Jamie Raskin tanked Doug Collins . Collins lo...
4,SARCASM,@USER @USER Pretty Sure the Anti-Lincoln Crowd...,[Man ... y ’ all gone “ both sides ” the apoca...


In [11]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
train_label=le.fit_transform(twitter_train['label'])
twitter_train['label']=train_label
twitter_train.head()

,label,response,context
0,1,@USER @USER @USER I don't get this .. obviousl...,[A minor child deserves privacy and should be ...
1,1,@USER @USER trying to protest about . Talking ...,[@USER @USER Why is he a loser ? He's just a P...
2,1,@USER @USER @USER He makes an insane about of ...,[Donald J . Trump is guilty as charged . The e...
3,1,@USER @USER Meanwhile Trump won't even release...,[Jamie Raskin tanked Doug Collins . Collins lo...
4,1,@USER @USER Pretty Sure the Anti-Lincoln Crowd...,[Man ... y ’ all gone “ both sides ” the apoca...


In [12]:
type(twitter_train)

pandas.core.frame.DataFrame

In [13]:
twitter_train['response']=twitter_train['response'].str.replace('@USER', "") 
twitter_train['response']=twitter_train['response'].str.replace('\d+', '')
twitter_train['response']=twitter_train['response'].str.lower()
twitter_train['response']=twitter_train['response'].str.replace('[^\w\s]','')

twitter_train['context']=twitter_train['context'].apply(lambda x: ','.join(map(str, x)))
twitter_train['context']=twitter_train['context'].str.replace('@USER', "") 
twitter_train['context']=twitter_train['context'].str.lower()
twitter_train['context']=twitter_train['context'].str.replace('[^\w\s]','')
twitter_train['context']=twitter_train['context'].str.replace('\d+', '')

twitter_train.head()

,label,response,context
0,1,i dont get this obviously you do care or y...,a minor child deserves privacy and should be k...
1,1,trying to protest about talking about him a...,why is he a loser hes just a press secretar...
2,1,he makes an insane about of money from the ...,donald j trump is guilty as charged the evid...
3,1,meanwhile trump wont even release his sat sc...,jamie raskin tanked doug collins collins look...
4,1,pretty sure the antilincoln crowd claimed th...,man y all gone both sides the apocalypse o...


In [14]:
twitter_train.shape

(5000, 3)

In [15]:

from collections import Counter
Counter(twitter_train['label'])


Counter({0: 2500, 1: 2500})

In [16]:
twitter_train['response'][0]

'   i dont get this  obviously you do care or you wouldve moved right along  instead you decided to care and troll her '

In [24]:
X=twitter_train['response'].values
y=twitter_train['label'].values
cont=twitter_train['context'].values

In [20]:
X.shape

(5000,)

In [22]:
y = pd.get_dummies(twitter_train['label'])
y = y.iloc[:,1].values
y

array([1, 1, 1, ..., 0, 0, 0], dtype=uint8)

In [23]:
from sklearn.model_selection import train_test_split
training_sentences,testing_sentences,training_labels,testing_labels = train_test_split(X, y, test_size=0.2, random_state=0)

"@USER @USER @USER I don't get this .. obviously you do care or you would've moved right along .. instead you decided to care and troll her .."

In [25]:
#change this

training_cont=cont[0:training_size]
testing_cont=cont[training_size:]

In [26]:
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
tokenizer.fit_on_texts(training_cont)
word_index = tokenizer.word_index

training_sequences = tokenizer.texts_to_sequences(training_sentences)
training_padded = pad_sequences(training_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
train_cont_sequences=tokenizer.texts_to_sequences(training_cont)
training_cont_padded = pad_sequences(train_cont_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
test_cont_sequences=tokenizer.texts_to_sequences(testing_cont)
test_cont_padded = pad_sequences(test_cont_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)


In [27]:
import numpy as np
training_padded = np.array(training_padded)

training_cont_padded=np.array(training_cont_padded)
training_labels = np.array(training_labels)
testing_labels=np.array(testing_labels)
testing_padded=np.array(testing_padded)
test_cont_padded=np.array(test_cont_padded)

In [28]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=5000),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid'),
    tf.keras.layers.Dropout(0.5)
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [29]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 5000, 16)          16000000  
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 24)                408       
_________________________________________________________________
dropout (Dropout)            (None, 24)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 25        
_________________________________________________________________
dropout_1 (Dropout)          (None, 1)                 0         
Total params: 16,000,433
Trainable params: 16,000,433
Non-trainable params: 0
____________________________________________

In [30]:
arr=[1,2]

In [31]:
arr

[1, 2]

In [32]:
a=2
b=3
arr2=[a,b]

In [33]:
arr3=[arr,arr2]

In [34]:
arr3

[[1, 2], [2, 3]]

In [35]:
type(training_padded)

numpy.ndarray

In [36]:
num_epochs = 10
a=np.array([training_padded,training_cont_padded])
b=np.array([testing_padded,test_cont_padded])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  This is separate from the ipykernel package so we can avoid doing imports until


In [37]:
type(a)

numpy.ndarray

In [38]:
type(a)
type(b)

print(len(training_cont_padded))
print(len(training_padded))
print(len(test_cont_padded))
print(len(testing_padded))
print(len(training_labels))
print(len(testing_labels))


2500
4000
2500
1000
4000
1000


In [40]:

history=model.fit(training_padded,training_labels,epochs=10,validation_data=(testing_padded,testing_labels),verbose=2)

Epoch 1/10
125/125 - 20s - loss: 4.4139 - accuracy: 0.4950 - val_loss: 0.7501 - val_accuracy: 0.5190
Epoch 2/10
125/125 - 20s - loss: 4.3392 - accuracy: 0.4933 - val_loss: 0.7698 - val_accuracy: 0.5190
Epoch 3/10
125/125 - 20s - loss: 4.2500 - accuracy: 0.5052 - val_loss: 0.7878 - val_accuracy: 0.5190
Epoch 4/10
125/125 - 20s - loss: 4.2752 - accuracy: 0.4910 - val_loss: 0.7943 - val_accuracy: 0.5190
Epoch 5/10
125/125 - 20s - loss: 4.1946 - accuracy: 0.5185 - val_loss: 0.7881 - val_accuracy: 0.5190
Epoch 6/10
125/125 - 20s - loss: 4.2674 - accuracy: 0.5117 - val_loss: 0.7922 - val_accuracy: 0.5190
Epoch 7/10
125/125 - 20s - loss: 4.3034 - accuracy: 0.5107 - val_loss: 0.7874 - val_accuracy: 0.5190
Epoch 8/10
125/125 - 20s - loss: 4.1911 - accuracy: 0.5210 - val_loss: 0.7825 - val_accuracy: 0.5190
Epoch 9/10
125/125 - 20s - loss: 4.1606 - accuracy: 0.5362 - val_loss: 0.7816 - val_accuracy: 0.5190
Epoch 10/10
125/125 - 20s - loss: 4.1888 - accuracy: 0.5253 - val_loss: 0.7706 - val_accura

In [41]:
sentence = ["Well now thats problematic AF ", "My 5 year old ... asked me why they are making fun of Native Americans ..", "i will take shit that didn't happen for  100", " no .. he actually in the gifted program and reads on second grade level .  ... and he knows Kansas City is in Missouri"]
context=["doesnt even know easy sum","doesnt even know easy sum"]
sequences = tokenizer.texts_to_sequences(sentence)
cont_sequences=tokenizer.texts_to_sequences(context)
padded = pad_sequences(sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
padded_cont=pad_sequences(cont_sequences,max_length,padding=padding_type,truncating='post')
print(model.predict(padded))

[[0.283212  ]
 [0.28605753]
 [0.27892256]
 [0.27297598]]


In [42]:
import pandas as pd
twitter_test=pd.read_json('twitter_test.jsonl',lines='True')
twitter_test.head()

,context,response,id
0,"[Well now that ’ s problematic AF <URL>, @USER...","@USER @USER @USER My 3 year old , that just fi...",twitter_1
1,[Last week the Fake News said that a section o...,@USER @USER How many verifiable lies has he to...,twitter_2
2,[@USER Let ’ s Aplaud Brett When he deserves i...,@USER @USER @USER Maybe Docs just a scrub of a...,twitter_3
3,[Women generally hate this president . What's ...,@USER @USER is just a cover up for the real ha...,twitter_4
4,"[Dear media Remoaners , you excitedly sharing ...",@USER @USER @USER The irony being that he even...,twitter_5


In [43]:
twitter_test=twitter_test.drop(['id'],axis='columns')

In [44]:
twitter_test.head()

,context,response
0,"[Well now that ’ s problematic AF <URL>, @USER...","@USER @USER @USER My 3 year old , that just fi..."
1,[Last week the Fake News said that a section o...,@USER @USER How many verifiable lies has he to...
2,[@USER Let ’ s Aplaud Brett When he deserves i...,@USER @USER @USER Maybe Docs just a scrub of a...
3,[Women generally hate this president . What's ...,@USER @USER is just a cover up for the real ha...
4,"[Dear media Remoaners , you excitedly sharing ...",@USER @USER @USER The irony being that he even...


In [52]:
sentences=twitter_test['response'].values

In [53]:
sentences[0]

'@USER @USER @USER My 3 year old , that just finished reading Nietzsche and then asked me : " ayo papa why these people always trying to cancel someone on Twitter , trying to pretend like that makes them better themselves ? " . To which I replied " idk " , and he just " cuz hoes mad " . Im so proud . <URL>'

In [54]:
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(sentences)

word_index = tokenizer.word_index

result_sequences = tokenizer.texts_to_sequences(sentences)
result_padded = pad_sequences(result_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)


In [55]:
import numpy as np
result_padded = np.array(result_padded)


In [62]:
len(result_padded)

1800

In [56]:
model.predict(result_padded)

array([[0.29787323],
       [0.2797593 ],
       [0.27146894],
       ...,
       [0.27081504],
       [0.2679575 ],
       [0.2519898 ]], dtype=float32)

In [63]:
for i in range(1700,1720):
  print(model.predict(result_padded)[i])

[0.29157376]
[0.2717113]
[0.27542338]
[0.25970668]
[0.27670288]
[0.27018702]
[0.29078358]
[0.27850628]
[0.28367215]
[0.25138]
[0.2693352]
[0.25491846]
[0.2739408]
[0.26769227]
[0.2867635]
[0.27833712]
[0.28728837]
[0.2813533]
[0.2653113]
[0.28070742]


FINAL PROBABILITIES OF STATEMENT BEING SARCASTIC


USING CNN TO DETECT SARCASM

In [64]:
from tensorflow.keras.layers import Dense, Dropout, Embedding, Flatten
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix

In [65]:
from keras.models import Sequential
from keras.models import Model, load_model
from keras.layers import Input, Dense, Embedding, Conv1D, MaxPooling1D, Flatten
from keras import optimizers

In [66]:
input_data = Input(shape=(max_length,), name='main_input')
embedding_layer = Embedding(vocab_size, embedding_dim, input_length=max_length)(input_data)
conv_1 = Conv1D(filters=50, kernel_size=4, activation='relu')(embedding_layer)
tf.keras.layers.Dropout(0.5)
max_1 = MaxPooling1D(pool_size=2)(conv_1)
conv_2 = Conv1D(filters=100, kernel_size=3, activation='relu')(max_1)
tf.keras.layers.Dropout(0.5)
max_2 = MaxPooling1D(pool_size=2)(conv_2)
flatten = Flatten()(max_2)
dense = Dense(100, activation='relu', name='fully_connected')(flatten)
tf.keras.layers.Dropout(0.5)
out = Dense(1, activation='sigmoid')(dense)
tf.keras.layers.Dropout(0.5)
model_01 = Model(inputs=[input_data], outputs=[out])

print(model_01.summary())

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
main_input (InputLayer)      [(None, 100)]             0         
_________________________________________________________________
embedding (Embedding)        (None, 100, 16)           16000000  
_________________________________________________________________
conv1d (Conv1D)              (None, 97, 50)            3250      
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 48, 50)            0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 46, 100)           15100     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 23, 100)           0         
_________________________________________________________________
flatten (Flatten)            (None, 2300)              0     

In [67]:
opt = tf.keras.optimizers.Adam(learning_rate=0.0001)
model_01.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])

model_01.fit(x=training_padded, y=training_labels, batch_size=256,epochs=10, validation_data=(testing_padded,testing_labels), verbose=1)

Epoch 1/10
16/16 [==============================] - 20s 281ms/step - loss: 0.6926 - accuracy: 0.5181 - val_loss: 0.6909 - val_accuracy: 0.5790
Epoch 2/10
16/16 [==============================] - 4s 237ms/step - loss: 0.6899 - accuracy: 0.5946 - val_loss: 0.6871 - val_accuracy: 0.5990
Epoch 3/10
16/16 [==============================] - 4s 237ms/step - loss: 0.6853 - accuracy: 0.6085 - val_loss: 0.6824 - val_accuracy: 0.5910
Epoch 4/10
16/16 [==============================] - 4s 239ms/step - loss: 0.6784 - accuracy: 0.6091 - val_loss: 0.6760 - val_accuracy: 0.6000
Epoch 5/10
16/16 [==============================] - 4s 238ms/step - loss: 0.6701 - accuracy: 0.6181 - val_loss: 0.6705 - val_accuracy: 0.6030
Epoch 6/10
16/16 [==============================] - 4s 237ms/step - loss: 0.6638 - accuracy: 0.6211 - val_loss: 0.6680 - val_accuracy: 0.6010
Epoch 7/10
16/16 [==============================] - 4s 239ms/step - loss: 0.6599 - accuracy: 0.6264 - val_loss: 0.6669 - val_accuracy: 0.6020
Epoch